# FID 
using code from https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/

In [1]:
# example of calculating the frechet inception distance in Keras for cifar10
import os
# import cv2
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [2]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))

## calculate FID for images in cifar10 as the example

In [3]:
# load cifar10 images
(images1, _), (images2, _) = cifar10.load_data()
shuffle(images1)
images1 = images1[:1000]
images2 = images2[:1000]
print('Loaded', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

Loaded (1000, 32, 32, 3) (1000, 32, 32, 3)
Scaled (1000, 299, 299, 3) (1000, 299, 299, 3)


In [4]:
# calculate fid
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)

32/32 [==============================] - 34s 1s/step
FID: 49.042


## Load microscopy images

In [5]:
from PIL import Image
# list the images in augmented data
perc_train = 0.8
N = 1000

dirpath = os.getcwd()
data_path = os.path.join(dirpath, "augmented_data") # read some of the aumented data
img_list = numpy.array([filename.split(".jpg")[0] for filename in os.listdir(data_path)])
# split the data grouping by original image
group_list = numpy.array(["_".join(filename.split("_")[:2]) for filename in img_list])
group_list_un = numpy.unique(group_list)
num_train = numpy.round(len(group_list_un)*perc_train).astype('int')
numpy.random.seed(seed=2)
train_groups = numpy.random.choice(group_list_un, size=num_train, replace=False)
train_idx =sum([list(numpy.where(group_list == val)[0]) for val in train_groups], [])
test_idx = list(set(range(len(group_list))).difference(set(train_idx)))

train_im_list = img_list[train_idx]
test_im_list = img_list[test_idx]

train_imgs = [numpy.asarray(Image.open(os.path.join(data_path, filename + ".jpg"))) for filename in train_im_list]
test_imgs = [numpy.asarray(Image.open(os.path.join(data_path, filename + ".jpg"))) for filename in test_im_list]
rng = numpy.random.default_rng(42)
noise_imgs = rng.integers(0, 255, (N, 256, 256, 3)) # stack of different images

In [6]:
# resize images
images_train = scale_images(train_imgs, (299,299,3))
images_test = scale_images(test_imgs, (299,299,3))
images_noise = scale_images(noise_imgs, (299,299,3))
print('Scaled', images_train.shape, images_test.shape)
# pre-process images
images_train = preprocess_input(images_train)
images_test = preprocess_input(images_test)

Scaled (7680, 299, 299, 3) (1920, 299, 299, 3)


### calculate fid betweeen EM images

In [8]:
# calculate fid
fid = calculate_fid(model, images_train, images_test)
print('FID: %.3f' % fid)

60/60 [==============================] - 65s 1s/step
FID: 21.728


### calculate fid betweeen EM images and Cifar10

In [9]:
fid = calculate_fid(model, images_train, images2)
print('FID: %.3f' % fid)

32/32 [==============================] - 78s 2s/step
FID: 426.861


### calculate fid betweeen EM images and noise

In [10]:
fid = calculate_fid(model, images_train, images_noise)
print('FID: %.3f' % fid)

32/32 [==============================] - 38s 1s/step
FID: 18124.270
